In [1]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

ERROR! Session/line number was not unique in database. History logging moved to new session 141


In [2]:
os.listdir('data')

['bmr15315_1.out', 'bmr17160_1.out', 'bmr7339_1.out']

In [3]:
in_df = pd.read_csv('data/bmr15315_1.out', sep = ' ', names=['Residue', 'ShiftCript index'])

In [4]:
index_tensor = torch.tensor(in_df['ShiftCript index'])

In [5]:
windows = index_tensor.unfold(0, 6, 1)
# print(windows)

In [130]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
#         self.conv1 = nn.Conv1d(6, 36, 2)
        self.linear1 = nn.Linear(6, 10)
        self.linear2 = nn.Linear(10, 1)
        
    def forward(self, x):
#         x = F.max_pool1d(F.relu(self.conv1(x)),2)
        x = F.relu(self.linear1(x))
        print(x)
        x = self.linear2(x)
        return x
    
net = Net()
print(net)

Net(
  (linear1): Linear(in_features=6, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=1, bias=True)
)


In [32]:
# For some reason 1D convolution and I don't like each other

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 12, 3)  # (in_channels, out_channels, kernel_size)  # DON'T TOUCH FOR NOW
        self.linear1 = nn.Linear(12, 6)
        self.linear2 = nn.Linear(6, 1)
        
    def forward(self, x):
        x = F.max_pool1d(F.relu(self.conv1(x)),4)
        print(x)
#         print(x.size())
        x = x.view(-1, 12)  # Flatten!!! Eureka!!!
        print(x)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv1d(1, 12, kernel_size=(3,), stride=(1,))
  (linear1): Linear(in_features=12, out_features=6, bias=True)
  (linear2): Linear(in_features=6, out_features=1, bias=True)
)


In [6]:
# Now with 2 targets, and tuneable hyperparameters
number_of_kernels_conv = 42
# These 2 not fully understood. These values work. 
kernel_size = 3
pooling_window_size = 3

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, int(number_of_kernels_conv), int(kernel_size))  # (in_channels, out_channels, kernel_size)  # DON'T TOUCH FOR NOW
        self.linear1 = nn.Linear(int(number_of_kernels_conv * kernel_size / pooling_window_size) , 6)
        self.linear2 = nn.Linear(6, 1)
        self.linear_alpha = nn.Linear(int(number_of_kernels_conv * kernel_size / pooling_window_size),8)
        self.linear_beta = nn.Linear(8,4)
        self.linear_gamma = nn.Linear(4,1)
        
    def forward(self, x):
        x = F.max_pool1d(F.relu(self.conv1(x)), int(pooling_window_size))
#         print(x)
        print(x.size())
        x = x.view(-1, int(number_of_kernels_conv * kernel_size / pooling_window_size))  # Flatten!!! Eureka!!!
        print(x)
        y = F.relu(self.linear_alpha(x))
        y = F.relu(self.linear_beta(y))
        y = self.linear_gamma(y)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x, y
    
net = Net()
print(net)

Net(
  (conv1): Conv1d(1, 42, kernel_size=(3,), stride=(1,))
  (linear1): Linear(in_features=42, out_features=6, bias=True)
  (linear2): Linear(in_features=6, out_features=1, bias=True)
  (linear_alpha): Linear(in_features=42, out_features=8, bias=True)
  (linear_beta): Linear(in_features=8, out_features=4, bias=True)
  (linear_gamma): Linear(in_features=4, out_features=1, bias=True)
)


In [25]:
# Now with 2 targets, and tuneable hyperparameters
number_of_kernels_conv = 42
# These 2 not fully understood. These values work. 
kernel_size = 3
pooling_window_size = 3

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, int(number_of_kernels_conv), int(kernel_size))  # (in_channels, out_channels, kernel_size)  # DON'T TOUCH FOR NOW
        self.linear1 = nn.Linear(int(number_of_kernels_conv) , 6)
        self.linear2 = nn.Linear(6, 1)
        self.linear_alpha = nn.Linear(int(number_of_kernels_conv),8)
        self.linear_beta = nn.Linear(8,4)
        self.linear_gamma = nn.Linear(4,1)
        
    def forward(self, x):
        x = F.max_pool1d(F.relu(self.conv1(x)), int(pooling_window_size))
#         print(x)
        print(x.size())
        x = x.view(-1, int(number_of_kernels_conv))  # Flatten!!! Eureka!!!
        print(x)
        y = F.relu(self.linear_alpha(x))
        y = F.relu(self.linear_beta(y))
        y = self.linear_gamma(y)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x, y
    
net = Net()
print(net)
print(net(a))

Net(
  (conv1): Conv1d(1, 42, kernel_size=(3,), stride=(1,))
  (linear1): Linear(in_features=42, out_features=6, bias=True)
  (linear2): Linear(in_features=6, out_features=1, bias=True)
  (linear_alpha): Linear(in_features=42, out_features=8, bias=True)
  (linear_beta): Linear(in_features=8, out_features=4, bias=True)
  (linear_gamma): Linear(in_features=4, out_features=1, bias=True)
)
torch.Size([1, 42, 1])
tensor([[0.0000, 0.4292, 0.6018, 0.0000, 0.7921, 0.7279, 0.0000, 0.0000, 0.6149,
         0.5699, 0.5943, 0.0192, 0.5870, 0.3810, 0.6495, 0.2749, 0.5849, 0.6829,
         0.5876, 0.4669, 0.8300, 0.7069, 0.2250, 0.2040, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.1916, 0.4039, 0.8293, 0.0000, 0.3744, 0.1882,
         0.6047, 0.4449, 0.0000, 0.4885, 0.6169, 0.0873]],
       grad_fn=<ViewBackward>)
(tensor([[-0.3344]], grad_fn=<AddmmBackward>), tensor([[0.4727]], grad_fn=<AddmmBackward>))


In [109]:
# print(list(net.named_parameters()))

In [7]:
# a = windows[0].reshape(-1,1).unsqueeze(0)
a = windows[0]
print(a)

tensor([-10.0000,   0.6654,   0.5337,   0.6093,   0.9774,   0.6371])


In [13]:
a = torch.randn(1,1,6)  # (Batch size, channels, size of input)
print(a)

tensor([[[ 0.0519, -0.5038,  0.4446, -0.8503,  0.4368, -1.3708]]])


In [9]:
# windows

In [10]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
# print(params)

12
torch.Size([42, 1, 3])


In [22]:
net(a)

torch.Size([1, 42, 1])
tensor([[0.2130, 0.1564, 0.0773, 0.0000, 0.0000, 0.2072, 0.4524, 0.0000, 0.5631,
         0.9466, 0.5106, 0.0044, 0.2100, 0.1567, 0.7990, 1.0699, 0.7559, 0.3927,
         0.2945, 0.0000, 0.2120, 0.0000, 0.1497, 0.4350, 0.1879, 0.0000, 0.0972,
         0.5938, 0.2778, 0.7052, 0.0000, 0.0000, 0.7290, 0.0000, 0.0828, 0.3928,
         0.3999, 0.1800, 0.7754, 0.3331, 0.0000, 0.0440]],
       grad_fn=<ViewBackward>)


(tensor([[0.5315]], grad_fn=<AddmmBackward>),
 tensor([[0.1521]], grad_fn=<AddmmBackward>))